# Eval Ragas

## 0. DB

In [4]:
from dotenv import load_dotenv

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader

load_dotenv()

True

In [6]:
embedding = OpenAIEmbeddings(model='text-embedding-3-small')
samsung_vision_2024_path = '../data/Sustainability_report_2024_kr.pdf'

docs = PyPDFLoader(samsung_vision_2024_path).load()

print(len(docs))

83


In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(docs[:20])

print(len(chunks))

48


In [8]:
collection_name = 'eval_data'
db_path = '../vectorstore/eval_db'

# db save
eval_db = Chroma.from_documents(
    documents=chunks, 
    collection_name=collection_name, 
    persist_directory=db_path, 
    embedding=embedding
)

## 1. RAG

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [23]:
retriever = eval_db.as_retriever(
    search_kwargs = {'k': 5}
)

In [2]:
prompt = ChatPromptTemplate.from_messages([
    ('system', """
    주어진 컨텍스트만 근거로 간결하고 정확하게 답하도록 해라.
    # Context
    {context}
    """),
    ('human', '{question}')
])

model = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0
)

output_parser = StrOutputParser()

rag_chain = prompt | model | output_parser

NameError: name 'ChatPromptTemplate' is not defined

## 3. CHAIN

In [25]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

def format_docs(docs):
    return '\n\n---\n\n'.join([doc.page_content for doc in docs])

chain = (
    {'context' : RunnableLambda(lambda x : x['question']) |  retriever | format_docs,
    'question' : RunnablePassthrough()
    }
    | rag_chain
)

In [26]:
chain.invoke({'question' : '삼성전자의 전망은?'})

'삼성전자는 지속가능성을 사업경쟁력과 기술혁신의 원동력으로 삼아 새로운 도약과 지속 성장을 모색하며, 글로벌 공시규제에 맞춘 투명한 정보 공개와 환경·사회·경제적 리스크 관리에 최선을 다할 전망입니다.'

## 4. Evaluation

In [5]:
import pandas as pd

excel_path = 'report_2024_test.xlsx'
df = pd.read_excel(excel_path)
df.head()

,user_input,reference_contexts,reference,synthesizer_name
0,How company use renewable energy in 미국?,"[""지속가능한 미래를 위한 노력을 계속해 \n왔습니다. 2050년 탄소중립을 통해 ...",In the DX division aiming for carbon neutralit...,single_hop_specific_query_synthesizer
1,Could you please explain the role and signific...,"[""고충의 처리 원칙에 대한 기준을 수립하였고, 공급망 관리에 \n있어서는 비제조 ...","In March 2023, Samsung Electronics established...",single_hop_specific_query_synthesizer
2,Can you explane in detail what the 5가지 핵심가치 ar...,['삼성전자 지속가능경영보고서 2024\n05\nOur Company Appendi...,Samsung Electronics has established 5가지 핵심가치 (...,single_hop_specific_query_synthesizer
3,What does DS stand for in Samsung Electronics'...,['Our Company AppendixMateriality Assessment F...,"In Samsung Electronics, DS stands for Device S...",single_hop_specific_query_synthesizer
4,How Mobile eXperience business doing in sales ...,"['매출\n169조 9,923억 원\n영업이익\n14조 3,847억 원 네트워크\n...",The Mobile eXperience business reported sales ...,single_hop_specific_query_synthesizer


## 평가 과정
- user_input : 질문
- reference_contexts : 예상 답변을 만들기 위해 참고한 context
- reference : 예상 답변
***
- retriever_contexts : 실제 검색된 context
- response : 실제 답변

In [ ]:
retriever_contexts = []

for question in df['user_input']:
    docs = retriever.invoke(question)
    ctx_list = []
    
    for doc in docs:
        ctx_list.append(doc.page_content)
        
    retriever_contexts.append(ctx_list)

In [ ]:
retriever_contexts[0]

['삼성전자 지속가능경영보고서 2024\n14\nOur Company AppendixMateriality Assessment Facts & Figures PrinciplePlanet People\nDX부문 재생에너지 전환율\n재생에너지 확대\nDX부문은 2027년까지 모든 사업장의 사용 전력 100%를 재생에너지로 \n전환하는 목표를 수립하였고 글로벌 시장에서 가용한 조 달 수단을 \n활용하여 재생에너지 사용량을 지속 확대하고 있습니다. 또한 재생에너지 \n전력구매계약(PPA, Power Purchase Agreement)의 점진적인 확대를 \n통하여 장기적으로 재생에너지 공급원을 확보하고, 사업장 내 여유 부지에 \n태양광 발전 설비를 설치하는 등 재생에너지 조달의 질적, 양적 개선을 \n도모하고 있습니다.\n주요 지역별 재생에너지 현황\n미국   미국 지역은 건물 내 태양광 발전 설비 설치, 재생에너지 인증서\n(REC, Renewable Energy Certificate) 구입 등을 통해 재생 에너지 \n사용을 늘려왔습니다. 그 결과 미국 사업장은 2020년 재생에너지 100% \n전환을 달성했습니다.\n유럽   유럽 지역은 녹색요금제(Green Pricing)와 REC구매를 통해 2020\n년부터 사용 전력을 100% 재생에너지로 전환했습니다.\n인도   인도 제조사업장 태양광, 풍력 및 바이오매스 발전사업자와 \n재생에너지 PPA를 체결하고 REC 구매, 태양광 발전 시설 설치 등을 통해 \n2022년 제조사업장의 재생에너지 100% 전환을 달성했습니다.\n베트남/중국   베트남 제조사업장은 REC 구매, 중국 지역은 사업장 내 \n재생에너지 PPA(태양광) 도입 및 REC 구매를 통해 2022년 사용 전력을 \n100% 재생에너지로 전환했습니다. 향후 국가별 상황에 맞춰 재생에너지 \nPPA 발굴 및 확대를 추진할 계획입니다.\n중남미   브라질 제조사업장은 REC를 구매하여 2022년 재 생에너지 \n100% 전환을 완료하였습니다. 멕시코 제조사업

In [37]:
response = []

for question in df['user_input']:
    answer = chain.invoke({'question' : question})
    response.append(answer)

In [ ]:
response[:5]

In [38]:
df['response'] = response
df['retriever_contexts'] = retriever_contexts

In [40]:
df.to_excel('report_2024_test_result.xlsx', index=False)

In [5]:
from ragas import EvaluationDataset, evaluate
from ragas.metrics import Faithfulness, LLMContextRecall, FactualCorrectness
from ragas.llms import LangchainLLMWrapper
from ragas.llms.base import llm_factory
import ast
import pandas as pd

In [6]:
df = pd.read_excel('report_2024_test_result.xlsx')
df.head()

,user_input,reference_contexts,reference,synthesizer_name,response,retriever_contexts
0,How company use renewable energy in 미국?,"[""지속가능한 미래를 위한 노력을 계속해 \n왔습니다. 2050년 탄소중립을 통해 ...",In the DX division aiming for carbon neutralit...,single_hop_specific_query_synthesizer,삼성전자는 미국에서 건물 내 태양광 발전 설비 설치와 재생에너지 인증서(REC) 구...,['삼성전자 지속가능경영보고서 2024\n14\nOur Company Appendi...
1,Could you please explain the role and signific...,"[""고충의 처리 원칙에 대한 기준을 수립하였고, 공급망 관리에 \n있어서는 비제조 ...","In March 2023, Samsung Electronics established...",single_hop_specific_query_synthesizer,"Subsidiary company 희망별숲, established by Samsun...","[""접수된 고충의 처리 원칙에 대한 기준을 수립하였고, 공급망 관리에 \n있어서는 ..."
2,Can you explane in detail what the 5가지 핵심가치 ar...,['삼성전자 지속가능경영보고서 2024\n05\nOur Company Appendi...,Samsung Electronics has established 5가지 핵심가치 (...,single_hop_specific_query_synthesizer,주어진 컨텍스트에는 삼성전자가 글로벌 행동강령과 지속가능한 성장을 위해 설정한 5가...,['· 업종 간 협력\n· 기후 대응을 포함한 \nUN SDGs에 대한 기여\n...
3,What does DS stand for in Samsung Electronics'...,['Our Company AppendixMateriality Assessment F...,"In Samsung Electronics, DS stands for Device S...",single_hop_specific_query_synthesizer,DS stands for Device Solutions in Samsung Elec...,['삼성전자 지속가능경영보고서 2024\n19\nOur Company Appendi...
4,How Mobile eXperience business doing in sales ...,"['매출\n169조 9,923억 원\n영업이익\n14조 3,847억 원 네트워크\n...",The Mobile eXperience business reported sales ...,single_hop_specific_query_synthesizer,"2023년 삼성전자 Mobile eXperience 사업부문의 매출은 66조 5,9...",['삼성전자 지속가능경영보고서 2024\n05\nOur Company Appendi...


In [7]:
df['reference_contexts'] = df['reference_contexts'].apply(lambda x : ast.literal_eval(x)) # 문자열을 list로 변환
df['reference_contexts'].dtype

dtype('O')

In [8]:
df['retrieved_contexts'] = df['retriever_contexts']
df.drop(columns=['retriever_contexts'], inplace=True)
df.head()

,user_input,reference_contexts,reference,synthesizer_name,response,retrieved_contexts
0,How company use renewable energy in 미국?,[지속가능한 미래를 위한 노력을 계속해 \n왔습니다. 2050년 탄소중립을 통해 글...,In the DX division aiming for carbon neutralit...,single_hop_specific_query_synthesizer,삼성전자는 미국에서 건물 내 태양광 발전 설비 설치와 재생에너지 인증서(REC) 구...,['삼성전자 지속가능경영보고서 2024\n14\nOur Company Appendi...
1,Could you please explain the role and signific...,"[고충의 처리 원칙에 대한 기준을 수립하였고, 공급망 관리에 \n있어서는 비제조 분...","In March 2023, Samsung Electronics established...",single_hop_specific_query_synthesizer,"Subsidiary company 희망별숲, established by Samsun...","[""접수된 고충의 처리 원칙에 대한 기준을 수립하였고, 공급망 관리에 \n있어서는 ..."
2,Can you explane in detail what the 5가지 핵심가치 ar...,[삼성전자 지속가능경영보고서 2024\n05\nOur Company Appendix...,Samsung Electronics has established 5가지 핵심가치 (...,single_hop_specific_query_synthesizer,주어진 컨텍스트에는 삼성전자가 글로벌 행동강령과 지속가능한 성장을 위해 설정한 5가...,['· 업종 간 협력\n· 기후 대응을 포함한 \nUN SDGs에 대한 기여\n...
3,What does DS stand for in Samsung Electronics'...,[Our Company AppendixMateriality Assessment Fa...,"In Samsung Electronics, DS stands for Device S...",single_hop_specific_query_synthesizer,DS stands for Device Solutions in Samsung Elec...,['삼성전자 지속가능경영보고서 2024\n19\nOur Company Appendi...
4,How Mobile eXperience business doing in sales ...,"[매출\n169조 9,923억 원\n영업이익\n14조 3,847억 원 네트워크\nM...",The Mobile eXperience business reported sales ...,single_hop_specific_query_synthesizer,"2023년 삼성전자 Mobile eXperience 사업부문의 매출은 66조 5,9...",['삼성전자 지속가능경영보고서 2024\n05\nOur Company Appendi...


In [9]:
df['retrieved_contexts'] = df['retrieved_contexts'].apply(lambda x : ast.literal_eval(x)) # 문자열을 list로 변환
df['retrieved_contexts'].dtype

dtype('O')

In [10]:
eval_llm = llm_factory(model = "gpt-4.1-mini")
# df : 'user_input', 'retrieved_contexts', 'reference_contexts', 'response', 'reference' 형태를 맞춰줘야 함
dataset = EvaluationDataset.from_pandas(df) # 키값이 정해져 있으므로 임의로 정하면 오류
dataset

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'reference_contexts', 'response', 'reference'], len=100)

In [ ]:
scores = evaluate(
    dataset,
    metrics=[Faithfulness(), # 할루시네이션 확인
            LLMContextRecall(), # retriever 확인 
            FactualCorrectness()], # system 프롬프트 확인(답변 '형식' 등)
    llm = eval_llm
)

Evaluating:   4%|▍         | 13/300 [02:36<46:38,  9.75s/it]  Exception raised in Job[0]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4.1-mini in organization org-ifMd863rcCIXpEuEYtSj7yxy on tokens per min (TPM): Limit 200000, Used 198773, Requested 3196. Please try again in 590ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
Exception raised in Job[14]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4.1-mini in organization org-ifMd863rcCIXpEuEYtSj7yxy on tokens per min (TPM): Limit 200000, Used 199216, Requested 1126. Please try again in 102ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
Evaluating:   6%|▌         | 18/300 [02:59<30:09,  6.42s/it]Exception raised in Job[4]: TimeoutError()
Exception raised in Job[3]: TimeoutError

KeyboardInterrupt: 

Exception raised in Job[163]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4.1-mini in organization org-ifMd863rcCIXpEuEYtSj7yxy on tokens per min (TPM): Limit 200000, Used 199012, Requested 3766. Please try again in 833ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
Exception raised in Job[180]: AssertionError(LLM is not set)
Exception raised in Job[181]: AssertionError(set LLM before use)
Exception raised in Job[182]: AssertionError(LLM must be set)
Exception raised in Job[183]: AssertionError(LLM is not set)
Exception raised in Job[184]: AssertionError(set LLM before use)
Exception raised in Job[185]: AssertionError(LLM must be set)
Exception raised in Job[186]: AssertionError(LLM is not set)
Exception raised in Job[187]: AssertionError(set LLM before use)
Exception raised in Job[188]: AssertionError(LLM must be set)
Exception raised in Job[189]: Asserti

In [ ]:
scores